# Extract MODIS Site Data and Generate Samples
## MODIS Site Data
For each site in the LFMC sample data, extract the full time series of MODIS reflectance and snow-cover data, and save to CSV files. Note: if the output csv files already exist they are assumed to be correct and are not over-written. The data is gap-filled after being saved but before being used to create the MODIS sample data.
## MODIS Sample Data
For each sample, extract the timeseries MODIS reflectance data. The timeseries length is determined by the MODIS_TS_LENGTH value. The sample is rejected if the full timeseries cannot be extracted (start/end outside the full site time series) or the snow-cover data shows the pixel was snow-covered on the sampling date. The extracted MODIS data is combined to a single dataframe and saved. Another LFMC sample dataset containing only the valid samples is also created.
## Input Files
- `LFMC_sites.csv` and `LFMC_samples.csv` created by the `Extract Auxiliary Data.ipynb` notebook.

## Output Files
- The extracted sites data are created in `MODIS_DIR` and `SNOW_DIR` (by default located in `DATA_DIR/GEE_DIR`). Both directories contain a CSV file for each site.
- The extracted MODIS data for each samples and updated samples data are created in `FINAL_DIR` (by default the Datasets sub-directory of `DATA_DIR/EXTRACT_NAME`). File names include the time series length (i.e. 365days) of the extracted MODIS data. So, with the default settings they are `modis_365days.csv` and `samples_365days.csv`.

## Notes
1. This notebook should be run after running the `Extract Auxiliary Data.ipynb` notebook.
2. The generated name for `GEE_DIR` includes the projection and scale of the extracted GEE data. 
3. It will take about 8.5 hours to run if there are no existing site extracts.
4. 824 samples will not be extracted due to snow cover.
5. 1005 samples will not be extracted as they were collected before March 2001 (so less than 1 year MODIS data available).
6. There should be no invalid sites, but occasionally extraction from GEE will fail for a site. If this happens re-run the notebook (keep the existing site CSV files so they are not re-extracted).

In [1]:
import os
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import initialise
from timeseries_extractor import GeeTimeseriesExtractor
from data_extract_utils import get_sample_data, sort_key

### Program parameters and constants

GEE Parameters - current settings are to extract the data needed to reproduce 1st paper
- Reflectance product is MCD43A4 - daily reflectance using 8-day composites
- Snow cover product is MOD10A1 - daily snow cover as MOD10A2 is not in GEE.
- Scale set to use native MODIS resolution

In [2]:
EXTRACT_NAME = "LFMC_ensembles"

START_DATE = "2000-03-01"
END_DATE = "2019-01-01"  # Final day retrieved will be 2018-12-31

# MODIS time series constants
MODIS_TS_LENGTH = 365   # 384
MODIS_TS_OFFSET = 1
MODIS_TS_FREQ = 1

# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]
SNOW_PRODUCT = "MODIS/006/MOD10A1"
SNOW_BANDS = ["NDSI_Snow_Cover"]
# Extract data using default MODIS scale and projection
SCALE = 463.3127 #30
PROJ = "SR-ORG:6974"

EARLIEST_SAMPLE = datetime.strptime(START_DATE, '%Y-%m-%d') + timedelta(
    days=MODIS_TS_LENGTH * MODIS_TS_FREQ + MODIS_TS_OFFSET)

### Directories and Files

In [3]:
# Top-level data directory
DATA_DIR = "G:\\My Drive\\LFMC Data\\"

# Sub-directories for GEE extracts
GEE_DIR = f"GEE_{PROJ.replace(':', '-')}_{int(SCALE)}"
MODIS_DIR = os.path.join(DATA_DIR, GEE_DIR, "MCD43A4")
SNOW_DIR = os.path.join(DATA_DIR, GEE_DIR, "MOD10A1")

# Sub-directory for pre-processed extracts
EXTRACT_DIR = os.path.join(DATA_DIR, EXTRACT_NAME)
SAMPLE_DIR = os.path.join(EXTRACT_DIR, "Globe-LFMC")  # 
FINAL_DIR = os.path.join(EXTRACT_DIR, "Datasets")

# File Names
LFMC_SITES = os.path.join(SAMPLE_DIR, "LFMC_sites.csv")
SAMPLES_INPUT = os.path.join(SAMPLE_DIR, "LFMC_samples.csv")
SAMPLES_OUTPUT = os.path.join(FINAL_DIR, f"samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")
MODIS_OUTPUT = os.path.join(FINAL_DIR, f"modis_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")

# Create output directories if necessary
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR)
if not os.path.exists(MODIS_DIR):
    os.makedirs(MODIS_DIR)
if not os.path.exists(SNOW_DIR):
    os.makedirs(SNOW_DIR)
if not os.path.exists(FINAL_DIR):
    os.makedirs(FINAL_DIR)

## Main Processing
Connect to GEE

In [4]:
import ee
ee.Initialize()

Check if a sample was collected in snow conditions

In [5]:
def is_snow_sample(date_str, bands_df, snow_df):
    sample_date = datetime.strptime(date_str, '%Y-%m-%d')
    return snow_df[SNOW_BANDS[0]][sample_date] >= 10

### Generate MODIS sample data

For each site
- Get the reflectance and snow data
- Then get the sample data for each sample at the site

Note: This gets the data using the default MODIS scale and projection. To change the scale and/or projection, add calls to the `GeeTimeseriesExtractor.setProjScale` method

In [6]:
sites = pd.read_csv(LFMC_SITES, float_precision="high")
samples = pd.read_csv(SAMPLES_INPUT, float_precision="high")
modis_extractor = GeeTimeseriesExtractor(PRODUCT, BANDS, START_DATE, END_DATE,
                                         dir_name=MODIS_DIR)
snow_extractor = GeeTimeseriesExtractor(SNOW_PRODUCT, SNOW_BANDS, START_DATE, END_DATE,
                                        dir_name=SNOW_DIR)
modis_data = []
valid_data = [False] * samples.shape[0]
invalid_pixels = []
snow_pixels = []
invalid_sites = []
for site_idx, site in sites.iterrows():
    print(f'Processing site {site.Site}')
    site_samples = samples[samples.Site == site.Site]
    try:
        modis_df = modis_extractor.get_and_save_data(site)
        snow_df = snow_extractor.get_and_save_data(site)
    except:
        print(f'Failed to extract data for {site.Site}')
        invalid_sites.append(site.Site)
        continue
    for index, sample in site_samples.iterrows():
        if is_snow_sample(sample["Sampling date"], modis_df, snow_df):
            print(f'Snow pixel: {sample["Sampling date"]}')
            snow_pixels.append(index)
        else:
            sample_data = get_sample_data(sample["Sampling date"], modis_df,
                                          MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)
            if sample_data is None or np.isnan(sample_data.sum()):
                invalid_pixels.append(index)
            else:
                modis_data.append([sample.ID] + list(sample_data))
                valid_data[index] = True

Processing site C4_1
Processing site C4_2
Processing site C4_3
Processing site C4_4
Processing site C4_5
Processing site C6_1
Snow pixel: 2012-08-15
Processing site C6_2
Processing site C6_3
Processing site C6_4
Processing site C6_5
Processing site C6_6
Processing site C6_7
Processing site C6_8
Processing site C6_9
Processing site C6_10
Processing site C6_11
Invalid date: 2000-04-07 outside valid date range
Invalid date: 2000-04-21 outside valid date range
Invalid date: 2000-05-05 outside valid date range
Invalid date: 2000-05-19 outside valid date range
Invalid date: 2000-06-02 outside valid date range
Invalid date: 2000-06-16 outside valid date range
Invalid date: 2000-06-30 outside valid date range
Invalid date: 2000-07-14 outside valid date range
Invalid date: 2000-07-28 outside valid date range
Processing site C6_12
Processing site C6_13
Processing site C6_14
Processing site C6_16
Processing site C6_17
Processing site C6_18
Processing site C6_19
Invalid date: 2000-03-14 outside va

Summary of sites/samples not extracted

In [7]:
print(f'Invalid sites: {len(invalid_sites)}; Invalid pixels: {len(invalid_pixels)}; ' \
      + f'Snow pixels: {len(snow_pixels)}')
print(invalid_sites)
print(invalid_pixels)
print(snow_pixels)

Invalid sites: 0; Invalid pixels: 1005; Snow pixels: 824
[]
[1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 6253, 6254, 6255, 6256, 6257, 6258, 6259, 6260, 6261, 6262, 14025, 14026, 14027, 14028, 14029, 14030, 14031, 14032, 14033, 14034, 14035, 14036, 16554, 16565, 16576, 16587, 16597, 16603, 16609, 22595, 22596, 22597, 22598, 22599, 22600, 22601, 22602, 22603, 22604, 22605, 28366, 28368, 28369, 28499, 28501, 28502, 28503, 28504, 28505, 28506, 28507, 35452, 35453, 35465, 35476, 35568, 35679, 38844, 38852, 39997, 39998, 39999, 40000, 48122, 48123, 48124, 48125, 48126, 49177, 49178, 49179, 49180, 49181, 49182, 50287, 50288, 50289, 50290, 50291, 67010, 67018, 67029, 67040, 67051, 67062, 67070, 67077, 67088, 67099, 68486, 68487, 68488, 68489, 68490, 68491, 68492, 68493, 68494, 68495, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1912, 1926, 1933, 1939, 5045, 5052, 5060, 5068, 5075, 5085, 5094, 6084, 6095, 6106, 6117, 6128, 6139, 6150, 6159

### Save Results
Save and display sample reflectance data

In [8]:
modis_data = pd.DataFrame(modis_data)
ts_days = (MODIS_TS_LENGTH - 1) * MODIS_TS_FREQ
modis_data.columns = ["ID"] + [f'{day-MODIS_TS_OFFSET:04}_{band+1}'
                               for day in range(-ts_days, 1, MODIS_TS_FREQ)
                               for band in range(len(BANDS))]
modis_data.sort_values('ID', inplace=True, key=lambda x: x.apply(sort_key))
modis_data.to_csv(MODIS_OUTPUT, index=False)
modis_data

,ID,-365_1,-365_2,-365_3,-365_4,-365_5,-365_6,-365_7,-364_1,-364_2,...,-002_5,-002_6,-002_7,-001_1,-001_2,-001_3,-001_4,-001_5,-001_6,-001_7
0,C4_1_1,1760,2497,1060,1461,3243,3128,2848,1752,2485,...,3285,3182,2553,1701,2738,900,1408,3285,3203,2568
1,C4_1_2,1757,2460,1060,1457,3251,3469,2965,1883,2640,...,3517,3526,2816,1820,2847,1015,1488,3506,3523,2816
2,C4_1_3,1966,2809,1124,1599,3497,3705,3108,1983,2826,...,3439,3521,2841,1896,2833,1036,1504,3452,3528,2847
3,C4_1_4,2003,2837,1147,1617,3407,3549,3155,2038,2891,...,3436,3480,2765,1849,2728,1039,1472,3404,3462,2754
4,C4_1_5,1929,2710,1135,1580,3368,3381,3080,1922,2710,...,3297,3368,2683,1815,2702,976,1420,3287,3326,2642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66933,C13_4_14,490,1881,267,477,2053,1623,788,493,1888,...,2358,1922,981,652,2062,351,589,2421,1931,1005
66934,C13_4_15,538,1912,298,515,2163,1690,839,536,1907,...,2383,1840,948,625,2039,342,574,2369,1811,925
66935,C13_4_16,522,1846,287,491,2060,1665,801,524,1839,...,2294,1798,960,640,1977,345,562,2294,1805,961
66936,C13_4_17,500,1796,267,451,1953,1610,741,495,1785,...,2208,1492,830,601,1924,320,524,2197,1489,824


Save and display the valid samples

In [9]:
valid_samples = samples[valid_data].sort_values('ID', key=lambda x: x.apply(sort_key))
valid_samples.to_csv(SAMPLES_OUTPUT, index=False)
valid_samples

,ID,Latitude,Longitude,Sampling date,Sampling year,Land Cover,LFMC value,Site,Czone1,Czone2,Czone3,Day_sin,Day_cos,Long_sin,Long_cos,Lat_norm,Elevation,Slope,Aspect_sin,Aspect_cos
163,C4_1_1,40.21458,-112.21868,2005-06-20,2005,Shrubland,156.76300,C4_1,B,BS,BSk,-0.21352,0.97694,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
164,C4_1_2,40.21458,-112.21868,2005-07-05,2005,Shrubland,128.27700,C4_1,B,BS,BSk,0.04302,0.99907,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
165,C4_1_3,40.21458,-112.21868,2005-07-21,2005,Shrubland,92.48200,C4_1,B,BS,BSk,0.31311,0.94972,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
166,C4_1_4,40.21458,-112.21868,2005-08-08,2005,Shrubland,82.09300,C4_1,B,BS,BSk,0.58779,0.80902,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
167,C4_1_5,40.21458,-112.21868,2005-08-23,2005,Shrubland,78.95300,C4_1,B,BS,BSk,0.77488,0.63210,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,C13_4_14,46.89791,-113.43535,2012-08-28,2012,"Tree cover, needleleaved, evergreen, closed (>...",102.44207,C13_4,B,BS,BSk,0.83593,0.54884,-0.91751,-0.39771,0.76054,0.21083,0.07778,0.54464,0.83867
61,C13_4_15,46.89791,-113.43535,2012-09-04,2012,"Tree cover, needleleaved, evergreen, closed (>...",88.76436,C13_4,B,BS,BSk,0.89584,0.44438,-0.91751,-0.39771,0.76054,0.21083,0.07778,0.54464,0.83867
62,C13_4_16,46.89791,-113.43535,2012-09-11,2012,"Tree cover, needleleaved, evergreen, closed (>...",88.79382,C13_4,B,BS,BSk,0.94276,0.33347,-0.91751,-0.39771,0.76054,0.21083,0.07778,0.54464,0.83867
63,C13_4_17,46.89791,-113.43535,2012-09-18,2012,"Tree cover, needleleaved, evergreen, closed (>...",81.72345,C13_4,B,BS,BSk,0.97601,0.21772,-0.91751,-0.39771,0.76054,0.21083,0.07778,0.54464,0.83867
